# AI vs Human Text Classifier Workflow

This notebook demonstrates the full workflow: data loading, preprocessing, feature extraction with transformers, model training, evaluation, and LIME explainability.

## 1. Import Required Libraries
We will use pandas, numpy, scikit-learn, transformers, torch, and lime for this workflow.

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from transformers import AutoTokenizer, AutoModel
import torch
import lime
import lime.lime_tabular

## 2. Load and Explore the Sample Dataset

In [ ]:
# Load the sample dataset
df = pd.read_csv('../data/sample_data.csv')
df.head()

In [ ]:
# Encode labels
le = LabelEncoder()
df['label_enc'] = le.fit_transform(df['label'])
df[['label', 'label_enc']].drop_duplicates()

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label_enc'], test_size=0.2, random_state=42)
print(f"Train size: {len(X_train)}, Test size: {len(X_test)}")

## 3. Feature Extraction with BERT

In [ ]:
# Use BERT to extract features (CLS token)
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

def get_bert_features(texts, tokenizer, model, max_length=128):
    features = []
    for text in texts:
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding='max_length', max_length=max_length)
        with torch.no_grad():
            outputs = model(**inputs)
            cls_embedding = outputs.last_hidden_state[:, 0, :].numpy()
        features.append(cls_embedding.squeeze())
    return np.array(features)

X_train_bert = get_bert_features(X_train, tokenizer, model)
X_test_bert = get_bert_features(X_test, tokenizer, model)
X_train_bert.shape, X_test_bert.shape

## 4. Train a Classifier

In [ ]:
# Train logistic regression classifier
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_bert, y_train)


## 5. Evaluate the Model

In [ ]:
# Predict and evaluate
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

y_pred = clf.predict(X_test_bert)
y_proba = clf.predict_proba(X_test_bert)[:, 1]
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)
print(f"Accuracy: {acc:.2f}, F1: {f1:.2f}, ROC-AUC: {roc_auc:.2f}")

## 6. Explain Predictions with LIME

In [ ]:
# LIME explanation for a test instance
explainer = lime.lime_tabular.LimeTabularExplainer(
    X_train_bert,
    feature_names=[f'feat_{i}' for i in range(X_train_bert.shape[1])],
    class_names=le.classes_,
    discretize_continuous=True
)

idx = 0  # Index of test instance to explain
exp = explainer.explain_instance(X_test_bert[idx], clf.predict_proba, num_features=10)
exp.show_in_notebook()